# Using Chat Message History

In [1]:
from functools import partial
from typing import Any, Dict, Optional

from langchain.prompts.chat import SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.memory.chat_message_histories import SQLChatMessageHistory
from langchain.schema.runnable.chat_history import RunnableWithChatHistory


In [2]:
# Stateless chat

prompt = (
    SystemMessagePromptTemplate.from_template("You are a nice assistant.")
    + MessagesPlaceholder(variable_name="history")
    + "{question}"
)

openai = ChatOpenAI()

chain = prompt | openai | StrOutputParser()

chain.invoke({"question": "Hello, how are you?", "history": []})


"Hello! I'm an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"

In [3]:
# Stateful chat

chain_with_history = RunnableWithChatHistory(
    factory=partial(SQLChatMessageHistory, connection_string="sqlite:///test.db"),
    runnable=chain,
    input_key="question"
)


In [4]:
chain_with_history.invoke(
    {"question": "Hello, I'm Nuno"},
    {"configurable": {"session_id": "test"}}
)


'Hello Nuno! How can I assist you today? Is there something specific you would like to know or discuss?'

In [5]:
chain_with_history.invoke(
    {"question": "What is my name?"},
    {"configurable": {"session_id": "test"}}
)


'Your name is Nuno, as you mentioned earlier. Is there anything else you would like to know or discuss?'